In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential #type:ignore
from tensorflow.keras.layers import Dense , Dropout , Bidirectional , LSTM #type:ignore
from tensorflow.keras.preprocessing.sequence import pad_sequences #type:ignore
from tensorflow.keras.callbacks import EarlyStopping #type:ignore


In [2]:

try:
    df = pd.read_csv('poultry_health_data.csv')
    print('Data Loaded Successfully')
    print(df.head())
    print(df.info())
except FileNotFoundError:
    print(f"File not found. Please ensure poultry_health_data.csv is in the current directory.")
    exit()

Data Loaded Successfully
         date  flock_id  day_of_flock_cycle  number_of_birds_start  \
0  2024-01-01  FLOCK_01                  58                   9667   
1  2024-01-01  FLOCK_02                  68                   6645   
2  2024-01-01  FLOCK_03                 129                   6989   
3  2024-01-01  FLOCK_04                 117                   9350   
4  2024-01-01  FLOCK_05                  80                   9130   

   daily_mortality  avg_weight_g  feed_consumption_kg  \
0               15       2346.44               709.41   
1               36       1168.39               570.78   
2               34       1118.20               934.04   
3               48       2316.01              1403.11   
4               24       1155.49              1435.41   

   water_consumption_liters  shed_temperature_c  shed_humidity_percent  \
0                   1364.70               32.81                  52.49   
1                   1998.52               29.96                

In [3]:

df['date'] = pd.to_datetime(df['date'])
df = df.sort_values(by=['flock_id','date'])

features = [
    'number_of_birds_start', 'daily_mortality', 'avg_weight_g',
    'feed_consumption_kg', 'water_consumption_liters',
    'shed_temperature_c', 'shed_humidity_percent', 'ammonia_level_ppm'
]
target = 'disease_outbreak'

In [4]:
scaler = MinMaxScaler(feature_range=(0,1))
df[features] = scaler.fit_transform(df[features])

sequences = []
labels = []
look_back = 14


In [6]:
for flock_id in df['flock_id'].unique():
    flock_df = df[df['flock_id']== flock_id].copy()

    if len(flock_df) > look_back:
        for i in range(len(flock_df) - look_back):
            seq_x = flock_df[features].iloc[i:i + look_back].values
            seq_y = flock_df[target].iloc[i + look_back]

            sequences.append(seq_x)
            labels.append(seq_y)

if not sequences:
    print("No sequences generated. Please check the data and look_back value.")
    exit()

X = np.array(sequences)
y = np.array(labels) 

print(f"\n Shape of X : { X.shape }  ")
print(f' Shape of y : { y.shape }')


 Shape of X : (1680, 14, 8)  
 Shape of y : (1680,)


In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y) 

print(f"X_train shape: {X_train.shape}, y_train shape: {y_train.shape}")
print(f"X_test shape: {X_test.shape}, y_test shape: {y_test.shape}")


X_train shape: (1176, 14, 8), y_train shape: (1176,)
X_test shape: (504, 14, 8), y_test shape: (504,)


In [10]:
model_bi = Sequential([
    Bidirectional(LSTM(units=50, return_sequences=True), input_shape=(X_train.shape[1], X_train.shape[2])),
    Dropout(0.2),
    Bidirectional(LSTM(units=100)),
    Dropout(0.2),
    Dense(units=64, activation='relu'),
    Dropout(0.2),
    Dense(units=1, activation='sigmoid')
])
model = model_bi 


c:\Users\Sarthak\OneDrive\Documents\Desktop\Python projects\farm-project\.venv\Lib\site-packages\keras\src\layers\rnn\bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [11]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ bidirectional (Bidirectional)   │ (None, 14, 100)        │        23,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 14, 100)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_1 (Bidirectional) │ (None, 200)            │       160,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 200)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 64)             │        12,864 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 197,329 (770.82 KB)

 Trainable params: 197,329 (770.82 KB)

 Non-trainable params: 0 (0.00 B)

In [14]:
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

print("\nTraining the model...")
history = model.fit(
    X_train, y_train,
    epochs=50, 
    batch_size=32,
    validation_split=0.1,
    callbacks=[early_stopping],
    verbose=1
)
print("Model training complete.")



Training the model...
Epoch 1/50


34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step - accuracy: 0.9903 - loss: 0.0594 - val_accuracy: 0.9915 - val_loss: 0.0538
Epoch 2/50
34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - accuracy: 0.9899 - loss: 0.0645 - val_accuracy: 0.9915 - val_loss: 0.0534
Epoch 3/50
34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - accuracy: 0.9885 - loss: 0.0653 - val_accuracy: 0.9915 - val_loss: 0.0555
Epoch 4/50
34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - accuracy: 0.9834 - loss: 0.0952 - val_accuracy: 0.9915 - val_loss: 0.0489
Epoch 5/50
34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - accuracy: 0.9813 - loss: 0.0952 - val_accuracy: 0.9915 - val_loss: 0.0493
Epoch 6/50
34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 29ms/step - accuracy: 0.9851 - loss: 0.0826 - val_accuracy: 0.9915 - val_loss: 0.0502
Epoch 7/50
34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - accuracy: 0.9824 - loss: 0.0935 - val_accuracy: 0.9915 - val_loss: 0.0499
Epoch 8/50
34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - accuracy: 0.9860 - loss: 0.0747 - val_accuracy: 0.9915 - val_loss: 0.

In [19]:
loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"\nTest Loss: {loss:.4f}")
print(f"Test Accuracy: {100 * accuracy:.4f}%")



Test Loss: 0.0741
Test Accuracy: 98.6111%


In [20]:
predictions_proba = model.predict(X_test)
predictions_class = (predictions_proba > 0.5).astype(int)

print("\nSample Predictions:")
for i in range(10):
    print(f"Actual: {y_test[i]}, Predicted Probability: {predictions_proba[i][0]:.4f}, Predicted Class: {predictions_class[i][0]}")


 1/16 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step

16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step

Sample Predictions:
Actual: 0, Predicted Probability: 0.0096, Predicted Class: 0
Actual: 0, Predicted Probability: 0.0115, Predicted Class: 0
Actual: 0, Predicted Probability: 0.0100, Predicted Class: 0
Actual: 0, Predicted Probability: 0.0089, Predicted Class: 0
Actual: 0, Predicted Probability: 0.0090, Predicted Class: 0
Actual: 0, Predicted Probability: 0.0102, Predicted Class: 0
Actual: 0, Predicted Probability: 0.0087, Predicted Class: 0
Actual: 0, Predicted Probability: 0.0101, Predicted Class: 0
Actual: 0, Predicted Probability: 0.0105, Predicted Class: 0
Actual: 0, Predicted Probability: 0.0104, Predicted Class: 0


In [21]:
import joblib 

joblib.dump(model,'poultry_health_data.joblib')
print('\n Model saved as poultry_health_data.joblib')


 Model saved as poultry_health_data.joblib
